<a href="https://colab.research.google.com/github/wilberquito/NeuralNetworksCompetition2023/blob/main/Code/ML_STUDENT_COMPETITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 500)
from google.colab import data_table
data_table.DataTable.max_columns = 50

data_table.enable_dataframe_formatter()

csv_path = Path('/content/drive/MyDrive/train.csv')
df = pd.read_csv(csv_path)
df.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,493553,0.315471,-0.183690,0.664383,-1.186794,0.665098,0.946208,0.729857,0,4,1,3,1,2,4,1,5,2,0,1,-3.690715,-0.628005,-2.832295,-1.409039,3.645067,0.233039,-3.754846,-1.061733,BDBBCACIBB,20.308715,1,0,0
1,237346,-1.286392,1.780592,0.576698,-2.690658,1.321997,-0.675894,0.371070,3,0,3,3,5,3,2,0,1,6,0,1,0.664517,-2.871912,3.826628,3.087653,0.494209,3.210875,-0.666457,0.123854,ACBDCBCADA,-449.291063,1,0,0
2,37368,-0.290303,-0.485907,0.808350,-0.156288,1.083632,-1.129914,0.767396,3,1,3,2,3,4,1,1,1,0,2,6,-0.494988,-0.946303,2.333223,2.084169,-4.782668,-1.671375,2.774382,2.273130,AABBABCLAF,-86.206118,0,1,1
3,665220,1.243590,0.035112,-1.013236,0.854267,0.019192,0.597892,-2.020416,2,0,4,5,0,5,1,0,3,1,1,3,-3.066427,-2.430158,-0.185332,-0.701691,-2.769142,-6.534231,-0.557677,-0.429972,ADBBABEEBA,-30.157403,0,2,1
4,41499,0.702716,0.817044,-0.064907,-1.045483,0.718374,0.164451,-0.936620,1,2,2,2,2,5,0,3,1,1,2,4,-1.899984,1.427460,-4.992610,1.154162,-1.931443,2.325042,2.143811,-1.039599,ABBBBBCMBB,296.484562,0,2,1


In [40]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [41]:
X, y = df.drop(['target', 'f_27', 'id'], axis=1), df['target']
X = (X - X.min()) / (X.max() - X.min())
X.shape, y.shape

((810000, 30), (810000,))

In [42]:
X.describe()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
count,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000
mean,0.492047,0.493137,0.483409,0.509344,0.489579,0.488666,0.500947,0.135472,0.128624,0.168756,0.155560,0.138772,0.177645,0.186609,0.108244,0.150097,0.139767,0.154813,0.158921,0.517581,0.487353,0.496712,0.510560,0.486064,0.465154,0.494703,0.538599,0.499857,0.345565,0.501186
std,0.106824,0.105223,0.104209,0.110100,0.103120,0.102858,0.103412,0.110450,0.099419,0.116962,0.117537,0.118276,0.110167,0.128143,0.097128,0.112068,0.103999,0.122292,0.120430,0.103392,0.105602,0.102610,0.105636,0.099971,0.100273,0.098855,0.091013,0.097074,0.475553,0.409425
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.419874,0.421923,0.413105,0.435033,0.419932,0.419381,0.431191,0.066667,0.062500,0.071429,0.071429,0.076923,0.125000,0.083333,0.000000,0.071429,0.066667,0.083333,0.076923,0.448633,0.415828,0.428052,0.440037,0.418822,0.397385,0.428586,0.479097,0.435201,0.000000,0.000000
50%,0.492222,0.493239,0.483497,0.509273,0.489467,0.488607,0.500879,0.133333,0.125000,0.142857,0.142857,0.153846,0.187500,0.166667,0.071429,0.142857,0.133333,0.166667,0.153846,0.518558,0.486864,0.496870,0.512280,0.485208,0.465171,0.494076,0.540384,0.499806,0.000000,0.500000
75%,0.564173,0.564118,0.553837,0.583519,0.559059,0.558114,0.570758,0.200000,0.187500,0.214286,0.214286,0.230769,0.250000,0.250000,0.142857,0.214286,0.200000,0.250000,0.230769,0.587767,0.558750,0.565416,0.582559,0.552316,0.532749,0.560920,0.600025,0.564667,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
y.value_counts()

0    415945
1    394055
Name: target, dtype: int64

In [44]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [45]:
class BinaryClassifier(nn.Module):

  def __init__(self, input_features=30):
    super().__init__()
    self.main = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=128),
        nn.ReLU(),
        nn.Linear(in_features=64, out_features=64),
        nn.ReLU(),
        nn.Linear(in_features=64, out_features=64),
        nn.ReLU(),
        nn.Linear(in_features=128, out_features=128),
        nn.ReLU(),
        nn.Linear(in_features=64, out_features=64),
        nn.ReLU(),
        nn.Linear(in_features=32, out_features=1),
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.main(x)

model = BinaryClassifier()
model

BinaryClassifier(
  (main): Sequential(
    (0): Linear(in_features=30, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ReLU()
    (10): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [49]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision


# Define a custom dataset by extending the PyTorch Dataset class
class CustomDataset(Dataset):

  def __init__(self, data, labels, transform=None):
    self.data = data
    self.labels = labels 
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    item = self.data[index]
    label = self.labels[index]
    if self.transform:
      item = self.transform(item)
    return item, label


In [61]:
# Declare the hole dataset
X_tensor, y_tensor = torch.from_numpy(X.to_numpy()), torch.from_numpy(y.to_numpy())
X_tensor, y_tensor = X_tensor.type(torch.float), y_tensor.type(torch.float)
print(X_tensor.shape, y_tensor.shape)
dataset = CustomDataset(data=X_tensor, labels=y_tensor)

torch.Size([810000, 30]) torch.Size([810000])


TypeError: ignored

In [51]:
# Declare the train and test dataset
import numpy as np

test_size = 0.2
train_size = 1 - test_size

# Stratify by label
labels = np.array(y)
positive_indices = np.where(labels == 1)[0]
negative_indices = np.where(labels == 0)[0]

positive_split = int(train_size * len(positive_indices))
negative_split = int(train_size * len(negative_indices))

positive_train_indices = positive_indices[:positive_split]
positive_test_indices = positive_indices[positive_split:]
negative_train_indices = negative_indices[:negative_split]
negative_test_indices = negative_indices[negative_split:]

train_indices = np.concatenate([positive_train_indices, negative_train_indices])
test_indices = np.concatenate([positive_test_indices, negative_test_indices])

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

batch_size = 64

# Create a DataLoader from the dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [62]:
for idx, (data, label) in enumerate(test_loader):
    print(idx, data.shape, label.shape)

0 torch.Size([64, 30]) torch.Size([64])
1 torch.Size([64, 30]) torch.Size([64])
2 torch.Size([64, 30]) torch.Size([64])
3 torch.Size([64, 30]) torch.Size([64])
4 torch.Size([64, 30]) torch.Size([64])
5 torch.Size([64, 30]) torch.Size([64])
6 torch.Size([64, 30]) torch.Size([64])
7 torch.Size([64, 30]) torch.Size([64])
8 torch.Size([64, 30]) torch.Size([64])
9 torch.Size([64, 30]) torch.Size([64])
10 torch.Size([64, 30]) torch.Size([64])
11 torch.Size([64, 30]) torch.Size([64])
12 torch.Size([64, 30]) torch.Size([64])
13 torch.Size([64, 30]) torch.Size([64])
14 torch.Size([64, 30]) torch.Size([64])
15 torch.Size([64, 30]) torch.Size([64])
16 torch.Size([64, 30]) torch.Size([64])
17 torch.Size([64, 30]) torch.Size([64])
18 torch.Size([64, 30]) torch.Size([64])
19 torch.Size([64, 30]) torch.Size([64])
20 torch.Size([64, 30]) torch.Size([64])
21 torch.Size([64, 30]) torch.Size([64])
22 torch.Size([64, 30]) torch.Size([64])
23 torch.Size([64, 30]) torch.Size([64])
24 torch.Size([64, 30]) to

In [ ]:
def binary_acc_computer(logits, objective):
  m = nn.Sigmoid()
  output = m(logits)
  return ((torch.sum(torch.round(output) == objective)) / len(objective)).item()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

X_train, y_train = X_train.to(device), y_train.to(device)  
X_test, y_test = X_test.to(device), y_test.to(device)  

model = model.to(device)

epochs = 100000
batch = 100

for epoch in range(1, epochs + 1):

  # Set model to train
  model.train()

  # Compute forward propagation
  logits_train = model(X_train).squeeze(dim=1)

  # Compute the training loss
  loss_train = criterion(logits_train, y_train)

  # Compute the training accuracy
  acc_train = binary_acc_computer(logits_train, y_train)

  # Reset the optimizer
  optimizer.zero_grad()

  # Compute the backpropagation
  loss_train.backward()

  # Modifies the parameters to optimize the function in the gradient direction
  optimizer.step()

  if epoch % batch == 0:
    print(f'Epoch: {epoch} | Train loss: {loss_train} | Train accuracy: {acc_train}')


In [ ]:
# Train
def train(device, model, epochs, optimizer, criteria, train_loader, valid_loader):
    # Early stopping
    the_last_loss = 100
    patience = 2
    trigger_times = 0

    for epoch in range(1, epochs+1):
        model.train()

        for times, data in enumerate(train_loader, 1):
            inputs = data[0].to(device)
            labels = data[1].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward and backward propagation
            outputs = model(inputs.view(inputs.shape[0], -1))
            loss = criteria(outputs, labels)
            loss.backward()
            optimizer.step()

            # Show progress
            if times % 100 == 0 or times == len(train_loader):
                print('[{}/{}, {}/{}] loss: {:.8}'.format(epoch, epochs, times, len(train_loader), loss.item()))

        # Early stopping
        the_current_loss = validation(model, device, valid_loader, loss_function)
        print('The current loss:', the_current_loss)

        if the_current_loss > the_last_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        else:
            print('trigger times: 0')
            trigger_times = 0

        the_last_loss = the_current_loss

    return model